# データローダー作成

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer


In [2]:
# データ読み込み
data = pd.read_pickle('sentiment_tech_data.pkl')
data

,open_mid,high_mid,low_mid,close_mid,pct_change,log_change,20SMA,20EMA,20ATR,20Upper,20Lower,Week,Year,US,JP,EU,CH,GE
datetime,,,,,,,,,,,,,,,,,,
2020-01-06 08:00:00,8772.55,8784.05,8747.70,8764.20,-0.005244,-0.005258,8790.7075,8796.991073,42.2625,8885.921797,8695.493203,1,2020,The U.S. stock market experienced a volatile f...,"The Japanese stock market, particularly the Ni...",During the first week of January 2020 (January...,"During the first week of 2020, the sentiment t...","In the first week of January 2020, the global ..."
2020-01-06 12:00:00,8764.20,8794.80,8763.00,8773.40,0.001050,0.001049,8792.7400,8794.744304,42.3625,8884.414743,8701.065257,1,2020,The U.S. stock market experienced a volatile f...,"The Japanese stock market, particularly the Ni...",During the first week of January 2020 (January...,"During the first week of 2020, the sentiment t...","In the first week of January 2020, the global ..."
2020-01-06 16:00:00,8773.40,8789.35,8723.35,8747.20,-0.002986,-0.002991,8792.6725,8790.216275,44.5200,8884.486124,8700.858876,1,2020,The U.S. stock market experienced a volatile f...,"The Japanese stock market, particularly the Ni...",During the first week of January 2020 (January...,"During the first week of 2020, the sentiment t...","In the first week of January 2020, the global ..."
2020-01-06 20:00:00,8747.20,8800.35,8729.05,8797.50,0.005750,0.005734,8795.1575,8790.909963,47.8850,8884.514892,8705.800108,1,2020,The U.S. stock market experienced a volatile f...,"The Japanese stock market, particularly the Ni...",During the first week of January 2020 (January...,"During the first week of 2020, the sentiment t...","In the first week of January 2020, the global ..."
2020-01-07 00:00:00,8797.50,8836.30,8792.60,8830.75,0.003779,0.003772,8799.5400,8794.704252,49.2550,8886.718278,8712.361722,1,2020,The U.S. stock market experienced a volatile f...,"The Japanese stock market, particularly the Ni...",During the first week of January 2020 (January...,"During the first week of 2020, the sentiment t...","In the first week of January 2020, the global ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31 12:00:00,14363.20,14365.85,14320.60,14350.70,-0.000870,-0.000871,14332.2925,14391.952249,92.1650,14424.624420,14239.960580,200,2023,The US stock market sentiment during the 44th ...,The Japanese stock market exhibited varied sen...,"During the 44th week of 2023, the European sto...","During the 44th week of 2023, Chinese stock ma...",The global economic outlook during the 44th we...
2023-10-31 16:00:00,14350.80,14438.30,14344.40,14425.70,0.005226,0.005213,14338.1350,14395.166321,95.4050,14438.646676,14237.623324,200,2023,The US stock market sentiment during the 44th ...,The Japanese stock market exhibited varied sen...,"During the 44th week of 2023, the European sto...","During the 44th week of 2023, Chinese stock ma...",The global economic outlook during the 44th we...
2023-10-31 20:00:00,14425.70,14453.90,14311.90,14417.20,-0.000589,-0.000589,14342.2000,14397.264766,97.0325,14448.727120,14235.672880,200,2023,The US stock market sentiment during the 44th ...,The Japanese stock market exhibited varied sen...,"During the 44th week of 2023, the European sto...","During the 44th week of 2023, Chinese stock ma...",The global economic outlook during the 44th we...


### 最大トークンサイズ

In [44]:
# # tokenizer インスタンスの生成
# from tqdm import tqdm

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# max_length = 0
# for region in tqdm(["US", "JP", "EU", "CH", "GE"]):
#     for i in tqdm(range(len(data))):
#         try:
#             token = tokenizer.tokenize(data[region][i])
#             max_length = len(token) if len(token) > max_length else max_length
#         except:
#             print(region, i, data[region][i])
# print(max_length)



  0%|          | 0/5 [00:00<?, ?it/s]/tmp/ipykernel_201863/2240130596.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  token = tokenizer.tokenize(data[region][i])
100%|██████████| 5/5 [02:30<00:00, 30.08s/it]

812


## Dataset

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# データ読み込み
data = pd.read_pickle('sentiment_tech_data.pkl')
data

class FinancialDataset(Dataset):
    def __init__(self, data, week_len=4, target_len=80):
        self.data = data
        self.week_len = week_len
        self.target_len = target_len
        self.x_tech, self.x_text, self.t = self.preprocessing(data, week_len, target_len)

    def __len__(self):
        return len(self.x_tech)

    def __getitem__(self, idx):
        return self.x_tech[idx], self.x_text[idx], self.t[idx]

    def preprocessing(self, data, week_len, target_len):
        target_len = int(target_len / 4) # 20
        x_tech_list = []
        x_text_list = []
        t_list = []

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        for week in range(1, data["Week"].max() + 1 - week_len):
            if not len(data[data["Week"] >= week + 4]) < target_len:
                target_data = data[data["Week"] >= week + 4].reset_index(drop=True)[:target_len]
            else:
                break

            target_data = target_data.reset_index(drop=True)
            target_data = target_data[["open_mid", "high_mid", "low_mid", "close_mid", "20SMA", "20Upper", "20Lower"]].values
            t_list.append(target_data)

            x_text_region_list = []
            week_4_data_list = []
            for i in range(week, week + 4):
                week_data = data[data["Week"] == i]
                x_tech = week_data.reset_index(drop=True)
                x_tech = x_tech.drop(["Week", "Year", "US", "JP", "EU", "CH", "GE"], axis=1)
                x_tech = x_tech.values
                x_tech = torch.tensor(x_tech, dtype=torch.float32)
                week_4_data_list.append(x_tech)

                # センチメントデータ
                ids_list = []
                for region in ["US", "JP", "EU", "CH", "GE"]:
                    sentiment = week_data[week_data["Week"] == i][region][0]
                    encoding = tokenizer(
                        sentiment, 
                        max_length=500, 
                        padding="max_length", 
                        truncation=True,
                        return_tensors="pt"
                    )
                    ids_list.append(encoding.input_ids)
                x_text_region = torch.stack(ids_list, dim=0)
                x_text_region_list.append(x_text_region)

            x_text_list.append(torch.stack(x_text_region_list, dim=0))
            x_tech_list.append(week_4_data_list)

        x_text = torch.stack(x_text_list, dim=0)
        t = torch.tensor(t_list, dtype=torch.float32)
        x_tech = x_tech_list
        return x_tech, x_text, t

    @staticmethod
    def collate_fn(batch):
        # バッチ内の各データポイントに対してパディングを適用
        x_tech_batch = [pad_sequence(week_data, batch_first=True) for week_data in zip(*[item[0] for item in batch])]
        x_tech_batch = torch.cat(x_tech_batch, dim=0).view(len(batch), -1, *x_tech_batch[0].shape[1:])

        x_text_batch = torch.stack([item[1] for item in batch])
        t_batch = torch.stack([item[2] for item in batch])

        return x_tech_batch, x_text_batch, t_batch

mydataset = FinancialDataset(data, 4, 80)
data_loader = DataLoader(mydataset, batch_size=16, shuffle=True, collate_fn=FinancialDataset.collate_fn)
for x_tech, x_text, t in data_loader:
    print(x_tech.shape, x_text.shape, t.shape)


/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipyke

torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torc

/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# データ読み込み
data = pd.read_pickle('sentiment_tech_data.pkl')
data

class FinancialDataset(Dataset):
    def __init__(self, data, week_len=4, target_len=80):
        self.data = data
        self.week_len = week_len
        self.target_len = target_len
        self.x_tech, self.x_text, self.t = self.preprocessing(data, week_len, target_len)

    def __len__(self):
        return len(self.x_tech)

    def __getitem__(self, idx):
        return self.x_tech[idx], self.x_text[idx], self.t[idx]

    def preprocessing(self, data, week_len, target_len):
        target_len = int(target_len / 4) # 20
        x_tech_list = []
        x_text_list = []
        t_list = []

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        for week in range(1, data["Week"].max() + 1 - week_len):
            if not len(data[data["Week"] >= week + 4]) < target_len:
                target_data = data[data["Week"] >= week + 4].reset_index(drop=True)[:target_len]
            else:
                break

            target_data = target_data.reset_index(drop=True)
            target_data = target_data[["open_mid", "high_mid", "low_mid", "close_mid", "20SMA", "20Upper", "20Lower"]].values
            t_list.append(target_data)

            x_text_region_list = []
            week_4_data_list = []
            for i in range(week, week + 4):
                week_data = data[data["Week"] == i]
                x_tech = week_data.reset_index(drop=True)
                x_tech = x_tech.drop(["Week", "Year", "US", "JP", "EU", "CH", "GE"], axis=1)
                x_tech = x_tech.values
                x_tech = torch.tensor(x_tech, dtype=torch.float32)
                week_4_data_list.append(x_tech)

                # センチメントデータ
                ids_list = []
                for region in ["US", "JP", "EU", "CH", "GE"]:
                    # sentiment = week_data[week_data["Week"] == i][region][0]
                    sentiment = week_data[week_data["Week"] == i].iloc[0][region]

                    encoding = tokenizer(
                        sentiment, 
                        max_length=850, 
                        padding="max_length", 
                        truncation=True,
                        return_tensors="pt"
                    )
                    ids_list.append(encoding.input_ids)
                x_text_region = torch.stack(ids_list, dim=0)
                x_text_region_list.append(x_text_region)

            x_text_list.append(torch.stack(x_text_region_list, dim=0))
            x_tech_list.append(week_4_data_list)

        x_text = torch.stack(x_text_list, dim=0)
        t = torch.tensor(t_list, dtype=torch.float32)
        x_tech = x_tech_list
        return x_tech, x_text, t

    @staticmethod
    def collate_fn(batch):
        # バッチ内の各データポイントに対してパディングを適用
        x_tech_batch = [pad_sequence(week_data, batch_first=True) for week_data in zip(*[item[0] for item in batch])]
        x_tech_batch = torch.cat(x_tech_batch, dim=0).view(len(batch), -1, *x_tech_batch[0].shape[1:])

        x_text_batch = torch.stack([item[1] for item in batch])
        t_batch = torch.stack([item[2] for item in batch])

        return x_tech_batch, x_text_batch, t_batch

mydataset = FinancialDataset(data, 4, 80)
data_loader = DataLoader(mydataset, batch_size=16, shuffle=True, collate_fn=FinancialDataset.collate_fn)
for x_tech, x_text, t in data_loader:
    print(x_tech.shape, x_text.shape, t.shape)


/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipyke

torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torch.Size([16, 4, 31, 11]) torch.Size([16, 4, 5, 1, 850]) torch.Size([16, 20, 7])
torc

/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipykernel_14725/1669941659.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentiment = week_data[week_data["Week"] == i][region][0]
/tmp/ipykernel_14725/1669941659.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400441250/work/torch/csrc/utils/tensor_new.cpp:261.